In [18]:
# import libraries
import robin_stocks.robinhood as r 
import pandas as pd
import numpy as np
import pyotp
import matplotlib.pyplot as plt
import datetime as dt

In [19]:
# Login robinhood

totp  = pyotp.TOTP("My2factorAppHere").now()
print("Current OTP:", totp)
r.login(username='mihir.argulkar@gmail.com',
         password='P%,fvd#C96XkCjE',
         expiresIn=120, mfa_code=totp)


# grabs our current stock portfolio
my_stocks = r.build_holdings()
# grab basic stats such as dividends and cash portfolio
user = r.build_user_profile()

# Define directory and filenames
export_dir = '/Users/mihirargulkar/robinhood-dashboard'
stock_orders_filename = 'stock_orders_Jan-20-2023.csv'
option_orders_filename = 'option_orders_Jan-20-2023.csv'

# grab history of sold stocks on the market
sold_stocks = r.export_completed_stock_orders(export_dir, stock_orders_filename)
# grab history of sold options on the market
sold_options = r.export_completed_option_orders(export_dir, option_orders_filename)

# create dataframes of our sold stocks and sold options
df_sold_stocks = pd.read_csv(stock_orders_filename)
df_sold_options = pd.read_csv(option_orders_filename)






Current OTP: 698950
Starting login process...
Found Additional pages.
Loading page 2 ...


In [178]:

option_orders = pd.read_csv('option_orders_Jan-20-2023.csv')
# change the order_created_at to datetime
option_orders['order_created_at'] = pd.to_datetime(option_orders['order_created_at'])
option_orders = option_orders.sort_values(by=['chain_symbol', 'expiration_date', 'order_created_at'], ascending=True)
option_orders = option_orders.drop(['order_type'], axis=1)

# Fill missing opening_strategy with closing_strategy, and vice versa
option_orders['opening_strategy'] = option_orders['opening_strategy'].fillna(option_orders['closing_strategy'])
option_orders['closing_strategy'] = option_orders['closing_strategy'].fillna(option_orders['opening_strategy'])
# Fix the creation of the 'id' column so that it is a string concatenation of the relevant columns, not a single string of the Series objects.
option_orders['id'] = (
    option_orders['chain_symbol'].astype(str) +
    option_orders['expiration_date'].astype(str) +
    option_orders['strike_price'].astype(str) +
    option_orders['option_type'].astype(str) +
    option_orders['opening_strategy'].astype(str) 
)

option_orders['id_leg'] = option_orders['direction'].astype(str) +  option_orders['id']

# If there are multiple rows with the same 'id_leg', drop the duplicates and increase the 'order_quantity' by the number of duplicates dropped
order_quantity = option_orders.groupby('id_leg').agg({'order_quantity': 'sum'}).reset_index()
option_orders['count'] = option_orders.groupby('id_leg')['id_leg'].transform('count')
option_orders['order_quantity'] = option_orders['order_quantity'] * option_orders['count']
option_orders = option_orders.drop(columns=['count'])


# # drop duplicate id_leg
option_orders = option_orders.drop_duplicates(subset=['id_leg'])




option_orders[option_orders['chain_symbol'].str.lower() == 'sofi']

,chain_symbol,expiration_date,strike_price,option_type,side,order_created_at,direction,order_quantity,opening_strategy,closing_strategy,price,processed_quantity,id,id_leg
200,SOFI,2024-05-03,7.0,put,sell,2024-04-11 17:09:18.195673+00:00,credit,1.0,short_put,short_put,0.36,1.0,SOFI2024-05-037.0putshort_put,creditSOFI2024-05-037.0putshort_put
140,SOFI,2025-02-07,15.0,put,sell,2025-01-28 14:35:54.112886+00:00,credit,1.0,short_put,short_put,0.46,1.0,SOFI2025-02-0715.0putshort_put,creditSOFI2025-02-0715.0putshort_put
139,SOFI,2025-02-07,15.0,put,buy,2025-01-28 15:01:02.850557+00:00,debit,1.0,short_put,short_put,0.30,1.0,SOFI2025-02-0715.0putshort_put,debitSOFI2025-02-0715.0putshort_put
132,SOFI,2025-03-07,13.5,put,sell,2025-02-28 14:33:06.806321+00:00,credit,1.0,short_put,short_put,0.37,1.0,SOFI2025-03-0713.5putshort_put,creditSOFI2025-03-0713.5putshort_put
131,SOFI,2025-03-07,13.5,put,buy,2025-02-28 14:49:22.664521+00:00,debit,1.0,short_put,short_put,0.27,1.0,SOFI2025-03-0713.5putshort_put,debitSOFI2025-03-0713.5putshort_put
130,SOFI,2025-03-14,13.5,put,sell,2025-03-03 17:08:10.329675+00:00,credit,1.0,short_put,short_put,0.28,1.0,SOFI2025-03-1413.5putshort_put,creditSOFI2025-03-1413.5putshort_put
127,SOFI,2025-03-28,13.0,call,sell,2025-03-17 15:29:10.403761+00:00,credit,1.0,short_call,short_call,0.22,1.0,SOFI2025-03-2813.0callshort_call,creditSOFI2025-03-2813.0callshort_call
125,SOFI,2025-04-11,13.5,call,sell,2025-04-02 17:17:04.455757+00:00,credit,1.0,short_call,short_call,0.15,1.0,SOFI2025-04-1113.5callshort_call,creditSOFI2025-04-1113.5callshort_call
123,SOFI,2025-05-02,15.0,call,sell,2025-04-23 13:30:53.939868+00:00,credit,1.0,short_call,short_call,0.08,1.0,SOFI2025-05-0215.0callshort_call,creditSOFI2025-05-0215.0callshort_call
121,SOFI,2025-05-09,12.5,put,buy,2025-04-28 13:35:15.088107+00:00,debit,1.0,long_put,long_put,0.65,1.0,SOFI2025-05-0912.5putlong_put,debitSOFI2025-05-0912.5putlong_put


In [166]:
# Concise processing for short puts and calls
short_trades = option_orders[option_orders['opening_strategy'].isin(['short_put', 'short_call'])].copy()
short_trades = short_trades.rename(columns={'chain_symbol': 'symbol', 'option_type': 'type', 'order_quantity': 'quantity'})
short_trades = short_trades.groupby('id').agg({
    'opening_strategy': 'first',
    'symbol': 'first',
    'strike_price': 'first',
    'expiration_date': 'first',
    'type': 'first',
    'quantity': 'first',
    'price': 'first',
    'direction': list
})

def get_exit_price(id_):
    group = option_orders[option_orders['id'] == id_]
    if (group['direction'] == 'debit').any():
        return group['price'].iloc[-1]
    return 0

short_trades['entry_price'] = short_trades['price']
short_trades['exit_price'] = short_trades.index.to_series().map(get_exit_price)
short_trades['pnl'] = (short_trades['entry_price'] - short_trades['exit_price']) * short_trades['quantity'] * 100
short_trades['cumulative_pnl'] = short_trades['pnl'].cumsum()
short_trades['collateral'] = short_trades['strike_price'] * short_trades['quantity'] * 100
short_trades['roi'] = ((short_trades['pnl'] / short_trades['collateral']) * 100).round(2)
short_trades = short_trades[[
    'opening_strategy', 'symbol', 'strike_price', 'expiration_date', 'type',
    'quantity', 'entry_price', 'exit_price', 'pnl', 'cumulative_pnl', 'collateral', 'roi'
]]

short_trades.shape

(92, 12)

In [162]:
# Concise processing for long puts and calls
long_trades = option_orders[option_orders['opening_strategy'].isin(['long_put', 'long_call'])].copy()
long_trades = long_trades.rename(columns={'chain_symbol': 'symbol', 'option_type': 'type', 'order_quantity': 'quantity'})
long_trades = long_trades.groupby('id').agg({
    'opening_strategy': 'first',
    'symbol': 'first',
    'strike_price': 'first',
    'expiration_date': 'first',
    'type': 'first',
    'quantity': 'first',
    'price': 'first',
    'direction': list
})

def get_exit_price(id_):
    group = option_orders[option_orders['id'] == id_]
    if (group['direction'] == 'credit').any():
        return group['price'].iloc[-1]
    return 0

long_trades['entry_price'] = long_trades['price']
long_trades['exit_price'] = long_trades.index.to_series().map(get_exit_price)
long_trades['pnl'] = (long_trades['exit_price'] - long_trades['entry_price']) * long_trades['quantity'] * 100
long_trades['cumulative_pnl'] = long_trades['pnl'].cumsum()
long_trades['collateral'] = long_trades['entry_price'] * long_trades['quantity'] * 100
long_trades['roi'] = ((long_trades['pnl'] / long_trades['collateral']) * 100).round(2)
long_trades = long_trades[[
    'opening_strategy', 'symbol', 'strike_price', 'expiration_date', 'type',
    'quantity', 'entry_price', 'exit_price', 'pnl', 'cumulative_pnl', 'collateral', 'roi'
]]
long_trades

,opening_strategy,symbol,strike_price,expiration_date,type,quantity,entry_price,exit_price,pnl,cumulative_pnl,collateral,roi
id,,,,,,,,,,,,
GOOGL2026-03-20130.0calllong_call,long_call,GOOGL,130.0,2026-03-20,call,1.0,35.00,46.00,1100.0,1100.0,3500.0,31.43
NKLA2024-08-306.5calllong_call,long_call,NKLA,6.5,2024-08-30,call,1.0,0.07,0.00,-7.0,1093.0,7.0,-100.00
QQQ2024-05-30452.0calllong_call,long_call,QQQ,452.0,2024-05-30,call,1.0,6.44,8.00,156.0,1249.0,644.0,24.22
QQQ2024-05-30453.0calllong_call,long_call,QQQ,453.0,2024-05-30,call,1.0,5.35,6.35,100.0,1349.0,535.0,18.69
SOFI2025-05-0912.5putlong_put,long_put,SOFI,12.5,2025-05-09,put,1.0,0.65,0.27,-38.0,1311.0,65.0,-58.46
TQQQ2024-07-2682.0calllong_call,long_call,TQQQ,82.0,2024-07-26,call,1.0,3.05,2.40,-65.0,1246.0,305.0,-21.31


In [159]:
# Short spreads (put/call): concise processing
ss_trades = option_orders[option_orders['opening_strategy'].isin(['short_put_spread', 'short_call_spread'])].copy()
ss_trades = ss_trades.rename(columns={'chain_symbol': 'symbol', 'order_quantity': 'quantity', 'option_type': 'type'})
group_idx = np.arange(len(ss_trades)) // 4
ss_trades['avg_strike'] = ss_trades['strike_price'].astype(float).groupby(group_idx).transform('mean')
ss_trades['id'] = (
    ss_trades['symbol'].astype(str) + "_" +
    ss_trades['avg_strike'].astype(str) + "_" +
    ss_trades['expiration_date'].astype(str) + "_" +
    ss_trades['type'].astype(str) + "_" +
    ss_trades['opening_strategy'].astype(str)
)
ss_trades = ss_trades.sort_values(['id', 'direction'])
group_cols = ['opening_strategy', 'symbol', 'avg_strike', 'expiration_date', 'type', 'id']
agg = {
    'strike_price': lambda x: list(dict.fromkeys(x)),
    'quantity': 'mean',
    'price': lambda x: list(dict.fromkeys(x)),
    'processed_quantity': 'mean',
    'direction': lambda x: list(dict.fromkeys(x))
}
ss_trades = ss_trades.groupby(group_cols, as_index=False).agg(agg)

def entry_exit(prices):
    if len(prices) == 2: return prices[0], prices[1]
    if len(prices) == 4: return prices[0] - prices[1], prices[2] - prices[3]
    return prices[0], prices[-1]

ss_trades['entry_price'], ss_trades['exit_price'] = zip(*ss_trades['price'].map(entry_exit))
ss_trades = ss_trades.set_index('id')

ss_trades['pnl'] = (ss_trades['entry_price'] - ss_trades['exit_price']) * 100 * ss_trades['quantity']
ss_trades['cumulative_pnl'] = ss_trades['pnl'].cumsum()
ss_trades['collateral'] = ss_trades.apply(
    lambda r: abs(float(r['strike_price'][0]) - float(r['strike_price'][1])) * 100 * r['quantity']
    if len(r['strike_price']) >= 2 else 0, axis=1
)
ss_trades['roi'] = (ss_trades['pnl'] / ss_trades['collateral'] * 100).round(2)

ss_trades = ss_trades[[
    'opening_strategy', 'symbol', 'strike_price', 'expiration_date', 'type',
    'quantity', 'price', 'entry_price', 'exit_price', 'pnl', 'cumulative_pnl', 'collateral', 'roi'
]]

ss_trades


,opening_strategy,symbol,strike_price,expiration_date,type,quantity,price,entry_price,exit_price,pnl,cumulative_pnl,collateral,roi
id,,,,,,,,,,,,,
AAPL_218.75_2025-08-01_call_short_call_spread,short_call_spread,AAPL,"[217.5, 220.0]",2025-08-01,call,1.0,"[0.74, 0.57]",0.74,0.57,17.0,17.0,250.0,6.8
AMD_131.0_2025-07-18_put_short_put_spread,short_put_spread,AMD,"[132.0, 130.0]",2025-07-18,put,1.0,"[0.54, 0.42]",0.54,0.42,12.0,29.0,200.0,6.0
GOOGL_162.5_2025-07-18_put_short_put_spread,short_put_spread,GOOGL,"[160.0, 165.0]",2025-07-18,put,1.0,"[1.27, 0.76]",1.27,0.76,51.0,80.0,500.0,10.2
GOOGL_168.75_2025-07-18_put_short_put_spread,short_put_spread,GOOGL,"[170.0, 167.5]",2025-07-18,put,1.0,"[0.48, 0.34]",0.48,0.34,14.0,94.0,250.0,5.6
HOOD_87.5_2025-07-18_put_short_put_spread,short_put_spread,HOOD,"[88.0, 87.0]",2025-07-18,put,1.0,"[0.32, 0.13]",0.32,0.13,19.0,113.0,100.0,19.0
NVDA_147.0_2025-07-18_put_short_put_spread,short_put_spread,NVDA,"[148.0, 146.0]",2025-07-18,put,1.0,"[0.43, 0.32]",0.43,0.32,11.0,124.0,200.0,5.5
SPY_589.0_2025-06-20_put_short_put_spread,short_put_spread,SPY,"[590.0, 588.0]",2025-06-20,put,1.0,"[0.24, 0.03]",0.24,0.03,21.0,145.0,200.0,10.5
TSLA_303.75_2025-07-03_put_short_put_spread,short_put_spread,TSLA,"[302.5, 305.0]",2025-07-03,put,1.0,"[0.5, 0.32]",0.50,0.32,18.0,163.0,250.0,7.2


In [175]:
final_col_order = [
    'opening_strategy', 'symbol', 'strike_price', 'expiration_date', 'type',
    'quantity', 'entry_price', 'exit_price', 'pnl', 'cumulative_pnl', 'collateral', 'roi'
]

# Ensure all columns are strings for set operations
short_cols = list(short_trades.columns)
long_cols = list(long_trades.columns)
ss_cols = list(ss_trades.columns)

# Use only columns that are hashable (avoid columns with list dtype)
def get_hashable_cols(df):
    # Exclude columns with dtype 'object' if they contain lists
    hashable_cols = []
    for col in df.columns:
        # Check if any value in the column is a list
        if df[col].apply(lambda x: isinstance(x, list)).any():
            continue
        hashable_cols.append(col)
    return hashable_cols

short_hashable = get_hashable_cols(short_trades)
long_hashable = get_hashable_cols(long_trades)
ss_hashable = get_hashable_cols(ss_trades)

all_cols = set(short_hashable) | set(long_hashable) | set(ss_hashable)
dfs = [short_trades, long_trades, ss_trades]
aligned = [df.reindex(columns=all_cols) for df in dfs]
combined_trades = pd.concat(aligned, axis=0, ignore_index=True)

# Now, for columns in final_col_order that are not in combined_trades, add them as NaN
for col in final_col_order:
    if col not in combined_trades.columns:
        combined_trades[col] = np.nan

# Reorder columns
combined_trades = combined_trades.reindex(columns=final_col_order)

# Check for duplicates, but only on hashable columns (exclude columns with lists)
# We'll use all columns except those with lists for duplicate checking
def is_list_col(series):
    return series.apply(lambda x: isinstance(x, list)).any()

hashable_for_dupes = [col for col in combined_trades.columns if not is_list_col(combined_trades[col])]
duplicates = combined_trades.duplicated(subset=hashable_for_dupes)
if duplicates.any():
    print(f"Found {duplicates.sum()} duplicate rows in combined_trades.")
    display(combined_trades[duplicates])
else:
    print("No duplicate rows found in combined_trades.")

if 'pnl' in combined_trades.columns:
    combined_trades['cumulative_pnl'] = combined_trades['pnl'].cumsum()
combined_trades

symbol_counts = combined_trades['symbol'].value_counts()
symbol_counts = symbol_counts.sort_index()
print("Count of each symbol (alphabetical order):")
print(symbol_counts)


No duplicate rows found in combined_trades.
Count of each symbol (alphabetical order):
symbol
AAL       1
AAPL      1
ACHR      1
AMD       1
ASTS     16
GOOGL     3
HIMS      1
HOOD      2
HUT       4
INTC      4
LUNR      1
MARA      6
MP        1
MRNA      6
NBIS      6
NKLA      4
NVDA      1
QQQ       2
RDFN      4
RGTI     10
RILY      5
RIVN      3
SMCI      7
SOFI     13
SPY       1
TQQQ      1
TSLA      1
Name: count, dtype: int64


In [180]:
combined_trades.to_csv("/Users/mihirargulkar/robinhood-dashboard/data/combined_trades.csv", index=False)
print("Exported combined_trades to combined_trades.csv")


Exported combined_trades to combined_trades.csv
